<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/main/notebooks/data_preprocessing_mem_reduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 14.0 MB/s 
     |████████████████████████████████| 182 kB 57.6 MB/s 
     |████████████████████████████████| 166 kB 62.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 71.9 MB/s 
     |████████████████████████████████| 162 kB 56.2 MB/s 
     |████████████████████████████████| 162 kB 59.9 MB/s 
     |████████████████████████████████| 158 kB 56.7 MB/s 
     |████████████████████████████████| 157 kB 55.5 MB/s 
     |████████████████████████████████| 157 kB 58.2 MB/s 
     |████████████████████████████████| 157 kB 60.0 MB/s 
     |████████████████████████████████| 157 kB 60.2 MB/s 
     |████████████████████████████████| 157 kB 68.3 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 70.5 MB/s 
     |███████████████████████████

In [2]:
# use gcs fuse to access google cloud storage
# https://stackoverflow.com/a/60450255/5755604
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# mount google cloud stoarge as drive
!mkdir gcs
!gcsfuse thesis-bucket-option-trade-classification gcs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  97576      0 --:--:-- --:--:-- --:--:-- 97576
OK
34 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 12.1 MB of archives.
After this operation, 27.6 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.7_amd64.deb ...
Unpacking gcsfuse (0.41.7) ...
Setting up gcsfuse (0.41.7) ...
2022/10/27 13:53:16.502948 Start gcsfuse/0.41.7 (Go version go1.18.4) for app "" using mount po

In [1]:
import numpy as np
# import pandas as pd
import modin.pandas as pd
import modin.config as cfg
from modin.config import ProgressBar
from distributed import Client
cfg.Engine.put("dask")
ProgressBar.enable()

import wandb

In [4]:
from google.colab import auth
auth.authenticate_user()


In [5]:
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
file_path_input = "gcs/data/raw/matched_ise_quotes.csv"
file_path_output = "gcs/data/preprocessed/matched_ise_quotes_min_mem_usage.pkl"

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage. 

	Adapted from: [load data (reduce memory usage) | Kaggle](https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook)
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    dfs = []
    chunksize = 10 ** 6
    with pd.read_csv(file, chunksize=chunksize, parse_dates=True, keep_date_col=True) as reader:
        for chunk in reader:
          dfs.append(chunk)

    df = pd.concat(dfs, axis=0)
    return reduce_mem_usage(df)

In [ ]:
client = Client()

df = import_data(file_path_input)
df.to_pickle(file_path_output)

dataset_at = wandb.Artifact('data_mem_reduced',type="preprocessed_data")
dataset_at.add_reference(file_path_output)


To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

To request implementation, send an email to feature_requests@modin.org.


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

...

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Memory usage of dataframe is 21772.90 MB


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Memory usage after optimization is: 7627.16 MB
Decreased by 65.0%
